In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install fastai==0.7.0 --no-deps

     |████████████████████████████████| 122kB 13.1MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import fastai

In [ ]:
!python prepare_data.py

In [ ]:
pip freeze | grep bcolz

In [ ]:
pip install fire 

     |████████████████████████████████| 92kB 6.5MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=fbd1eb0419dc2834df04edcf65af107ae7467f7ca353bb3094c73bdb833555d3
  Stored in directory: /root/.cache/pip/wheels/af/19/30/1ea0cad502dcb4e66ed5a690279628c827aea38bbbab75d5ed
Successfully built fire


In [ ]:
import torch
if torch.cuda.is_available:
  print('GPU available')
else:
  print('Please set GPU via Edit -> Notebook Settings.')

torch.cuda.get_device_name(0)

GPU available


'Tesla V100-SXM2-16GB'

In [ ]:
#pip install scipy==1.2.3

In [ ]:
import scipy
scipy.__version__

'1.2.3'

In [ ]:
#pip install numpy==1.17.5

In [ ]:
import numpy as np
np.__version__

'1.17.5'

In [ ]:
pip install -r requirements.txt

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.metrics import error_rate
from scipy.io import loadmat
import pandas as pd

In [ ]:
import fastai
fastai.__version__

'1.0.53.post2'

In [ ]:
## Training Data Cleaning
mat_data = loadmat('cars_train_annos.mat')
mat_meta = loadmat('cars_meta.mat')

class_id_label = mat_data['annotations']["class"].squeeze()
fname = mat_data['annotations']["fname"].squeeze()
class_names = mat_meta['class_names'].squeeze()

class_id_label_list = [class_id[0][0] for class_id in class_id_label]
fname_list = [f[0] for f in fname]
class_names_list = [c[0] for c in class_names]

# Mapping id to classnames for verboseness
class_name_label_list = [class_names_list[idx-1] for idx in class_id_label_list] 

train_df = pd.DataFrame(
    {'fname': fname_list,
     'label': class_name_label_list,
    })

In [ ]:
train_df.head()

,fname,label
0,00001.jpg,Audi TTS Coupe 2012
1,00002.jpg,Acura TL Sedan 2012
2,00003.jpg,Dodge Dakota Club Cab 2007
3,00004.jpg,Hyundai Sonata Hybrid Sedan 2012
4,00005.jpg,Ford F-450 Super Duty Crew Cab 2012


In [ ]:
from fastai.vision import *
tfms = get_transforms()  

In [ ]:
path = Path('/content/drive/MyDrive/ML_DL_Stuff/Concat_system/cars_train')
data = (
    ImageList
    .from_df(df=train_df, path=path)
    .split_by_rand_pct(valid_pct=0.2, seed=100) # Seed to make sure validation set is the same everytime we repeat an experiment
    .label_from_df(cols="label")
    .transform(tfms=tfms, size=299, resize_method=ResizeMethod.SQUISH, padding_mode='reflection')
    .databunch(bs=16) # Batch Size adjusted according to GPU Memory Capacity
    .normalize(imagenet_stats) # Normalize Image according to ImageNet's Normalization because we are using ImageNet's pretrained model
)

In [ ]:
data

ImageDataBunch;

Train: LabelList (6516 items)
x: ImageList
Image (3, 299, 299),Image (3, 299, 299),Image (3, 299, 299),Image (3, 299, 299),Image (3, 299, 299)
y: CategoryList
Audi TTS Coupe 2012,Acura TL Sedan 2012,Dodge Dakota Club Cab 2007,Hyundai Sonata Hybrid Sedan 2012,Ford F-450 Super Duty Crew Cab 2012
Path: /content/drive/MyDrive/ML_DL_Stuff/Concat_system/stanford-car-grab-challenge/cars_train;

Valid: LabelList (1628 items)
x: ImageList
Image (3, 299, 299),Image (3, 299, 299),Image (3, 299, 299),Image (3, 299, 299),Image (3, 299, 299)
y: CategoryList
Ferrari FF Coupe 2012,Rolls-Royce Ghost Sedan 2012,Isuzu Ascender SUV 2008,Dodge Charger SRT-8 2009,Chevrolet Sonic Sedan 2012
Path: /content/drive/MyDrive/ML_DL_Stuff/Concat_system/stanford-car-grab-challenge/cars_train;

Test: None

In [ ]:
test_path = '/content/drive/MyDrive/ML_DL_Stuff/Concat_system/cars_test'
test_mat_path = download_data(url='http://imagenet.stanford.edu/internal/car196/cars_test_annos_withlabels', ext=".mat")

In [ ]:
mat_test = loadmat('cars_test_annos.mat')
mat_test_label = loadmat(test_mat_path)

test_id_label = mat_test_label['annotations']["class"].squeeze()
test_fname = mat_test_label['annotations']["fname"].squeeze()

test_id_label_list = [class_id[0][0] for class_id in test_id_label]
test_fname_list = [f[0] for f in test_fname] 
class_names_list;

# Mapping id to classnames for verboseness
test_class_name_label_list = [class_names_list[idx-1] for idx in test_id_label_list] 

test_df = pd.DataFrame(
    {'fname': test_fname_list,
     'label': test_class_name_label_list,
    })

In [ ]:
test_df.head()

,fname,label
0,00001.jpg,Suzuki Aerio Sedan 2007
1,00002.jpg,Ferrari 458 Italia Convertible 2012
2,00003.jpg,Jeep Patriot SUV 2012
3,00004.jpg,Toyota Camry Sedan 2012
4,00005.jpg,Tesla Model S Sedan 2012


In [ ]:
df1 = train_df
df1["is_valid"] = False
df2 = test_df
df2["is_valid"] = True
real_test_df = pd.concat([df1,df2])

In [ ]:
real_test_df.head()

,fname,label,is_valid
0,00001.jpg,Audi TTS Coupe 2012,False
1,00002.jpg,Acura TL Sedan 2012,False
2,00003.jpg,Dodge Dakota Club Cab 2007,False
3,00004.jpg,Hyundai Sonata Hybrid Sedan 2012,False
4,00005.jpg,Ford F-450 Super Duty Crew Cab 2012,False


In [ ]:
test_data = (
    ImageList
    .from_df(df=real_test_df, path=test_path)
    .split_from_df()
    .label_from_df(cols="label")
    .transform(tfms=tfms, size=299, resize_method=ResizeMethod.SQUISH, padding_mode='reflection')
    .databunch(bs=16)
    .normalize(imagenet_stats)
)

In [ ]:
!python predict.py generate_csv_for_test_data --img_path=cars_test --output_fpath=test_1.csv

In [ ]:
test_df.to_csv('data_with_labels.csv', columns=test_df.columns)

In [ ]:
!python predict.py populate_csv_for_labelled_data --csv_path=data_with_labels.csv --img_path=cars_test --output_fpath=final.csv

Running Inference ...
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object